In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )
from math import sqrt

Carbon Version v1.5-beta0 (6/Dec/2022)


# Carbon Simulation -Demo 1-2

In demo we look at **single-curve unidirectional liquidity with a non-zero width range**

Set up the simulation. We do not provide a default curve, so curves must be specified every time.

In [2]:
Sim = CarbonSimulatorUI(verbose=False)
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair=None, mm='exact', xf=True)

## Adding and using up the first liquidity position

We set up a single curve where the AMM sells ETH against USDC in the range of 2000-3000. The geometric average is ~2449, which is the effective selling price of this range

In [3]:
Sim.add_order("ETH", 100, 2000, 3000, pair="ETH/USDC")["orders"]
print(sqrt(2000*3000))

2449.489742783178


We sell the entire ETH

In [4]:
result = Sim.amm_sells("ETH", 100, pair="ETH/USDC")
result["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,0,1,2449.489743,USDC per ETH
0,0,0,A,AMM sells 100ETH buys 244949USDC,True,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,[0],1,2449.489743,USDC per ETH


We are selling exactly where we expect to sell

In [5]:
result["trades"]["price"].iloc[0], sqrt(2000*3000)

(2449.489743, 2449.489742783178)

## Adding and using up the second liquidity position

We set up another single curve where the AMM sells ETH against USDC in the range of 2000-3000. The geometric average is ~2449, which is the effective selling price of this range. But first we check we have nothing left to sell.

In [6]:
Sim.amm_sells("ETH", 0.00001, pair="ETH/USDC")

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}

In [7]:
Sim.add_order("ETH", 100, 2000, 3000, pair="ETH/USDC")["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
1,1,ETHUSDC,ETH,100.0,100.0,ETH,2000.0,3000.0,2000.0,USDC per ETH,1


In [8]:
result = Sim.amm_sells("ETH", 100, pair="ETH/USDC")
result["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #1,False,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,1,1,2449.489743,USDC per ETH
0,1,1,A,AMM sells 100ETH buys 244949USDC,True,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,[1],1,2449.489743,USDC per ETH


In [9]:
result["trades"]["price"].iloc[0], sqrt(2000*3000)

(2449.489743, 2449.489742783178)

In [10]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,100.0,0.0,ETH,2000.0,3000.0,3000.0,USDC per ETH,0
1,1,ETHUSDC,ETH,100.0,0.0,ETH,2000.0,3000.0,3000.0,USDC per ETH,1


## Adding and using up the third liquidity position

We set up another single curve where the AMM sells ETH against USDC in the range of 2000-3000. The geometric average is ~2449, which is the effective selling price of this range. But first we check again that we have nothing left to sell.

In [11]:
Sim.amm_sells("ETH", 0.00001, pair="ETHUSDC")

{'success': False,
 'error': "('Illegal slashpair', 'ETHUSDC')",
 'exception': ValueError('Illegal slashpair', 'ETHUSDC')}

In [12]:
Sim.add_order("ETH", 100, 2000, 3000, pair="ETH/USDC")["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
2,2,ETHUSDC,ETH,100.0,100.0,ETH,2000.0,3000.0,2000.0,USDC per ETH,2


In [13]:
Sim.amm_sells("ETH", 10, pair="ETH/USDC")
Sim.amm_sells("ETH", 20, pair="ETH/USDC")
Sim.amm_sells("ETH", 70, pair="ETH/USDC")
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,100.0,0.0,ETH,2000.0,3000.0,3000.0,USDC per ETH,0
1,1,ETHUSDC,ETH,100.0,0.0,ETH,2000.0,3000.0,3000.0,USDC per ETH,1
2,2,ETHUSDC,ETH,100.0,0.0,ETH,2000.0,3000.0,3000.0,USDC per ETH,2


In [14]:
Sim.state()["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,0,1,2449.489743,USDC per ETH
0,0,0,A,AMM sells 100ETH buys 244949USDC,True,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,[0],1,2449.489743,USDC per ETH
0,1.0,1,0,route #1,False,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,1,1,2449.489743,USDC per ETH
0,1,1,A,AMM sells 100ETH buys 244949USDC,True,True,None,100.0,ETH,244948.974278,USDC,ETHUSDC,[1],1,2449.489743,USDC per ETH
0,2.0,2,0,route #2,False,True,None,10.0,ETH,20373.867433,USDC,ETHUSDC,2,1,2037.386743,USDC per ETH
0,2,2,A,AMM sells 10ETH buys 20374USDC,True,True,None,10.0,ETH,20373.867433,USDC,ETHUSDC,[2],1,2037.386743,USDC per ETH
0,3.0,3,0,route #2,False,True,None,20.0,ETH,43121.624528,USDC,ETHUSDC,2,1,2156.081226,USDC per ETH
0,3,3,A,AMM sells 20ETH buys 43122USDC,True,True,None,20.0,ETH,43121.624528,USDC,ETHUSDC,[2],1,2156.081226,USDC per ETH
0,4.0,4,0,route #2,False,True,None,70.0,ETH,181453.482317,USDC,ETHUSDC,2,1,2592.192605,USDC per ETH
0,4,4,A,AMM sells 70ETH buys 181453USDC,True,True,None,70.0,ETH,181453.482317,USDC,ETHUSDC,[2],1,2592.192605,USDC per ETH


In [15]:
prices = {
    10: Sim.state()["trades"].iloc[4]["price"],
    20: Sim.state()["trades"].iloc[6]["price"],
    70: Sim.state()["trades"].iloc[8]["price"],
}
prices

{10: 2037.386743, 20: 2156.081226, 70: 2592.192605}

In [16]:
usdc_total = sum(amt*prc for amt,prc in prices.items())
print(f"average price: {usdc_total/sum(prices.keys())}")

average price: 2449.489743
